<a href="https://colab.research.google.com/github/NULabTMN/homework-3-gmac98/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [2]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2022-04-09 22:15:25--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2022-04-09 22:15:25--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.1’

shakespeare_plays.t 100%[===================>]   4.53M  24.9MB/s    in 0.2s    

2022-04-09 22:15:26 (24.9 MB/s) - ‘shakespeare_plays.txt.1’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8294721841812134),
 ('prince', 0.7473874092102051),
 ('clarence', 0.7261805534362793),
 ('duke', 0.7200121879577637),
 ('york', 0.7096157073974609),
 ('gloucester', 0.7092002630233765),
 ('margaret', 0.6989930868148804),
 ('warwick', 0.6986757516860962),
 ('cardinal', 0.6986408233642578),
 ('princess', 0.6977097392082214)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [5]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9639438390731812),
 ('emilia', 0.9393848776817322),
 ('iago', 0.9315193295478821),
 ('countess', 0.9233903288841248),
 ('lucio', 0.9186805486679077),
 ('ham', 0.9124850034713745),
 ('imogen', 0.9101074934005737),
 ('rosalind', 0.9075084328651428),
 ('cleopatra', 0.9070605039596558),
 ('proteus', 0.9009823203086853)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [6]:
model.wv.similarity('othello', 'desdemona')

0.9639438

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [7]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  val1 = np.linalg.norm(v1)
  val2 = np.linalg.norm(v2)
  if val1 > 0 and val2 > 0:
        return np.dot(v1, v2) / (val1 * val2)
  else:
      return 0.0

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9639438

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [8]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2022-04-09 22:15:38--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2022-04-09 22:15:38--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M   107MB/s    in 0.6s    

2022-04-09 22:15:39 (107 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2022-04-09 22:15:39--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.2

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [9]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [10]:
## TODO: implement this search function
def mostSimilar(vec, vecDict, topn):

  ## Use your cosim function from above
  similarity = [(k, cosim(vec, v)) for k,v in vecDict.items()]
    
  return sorted(similarity, key = lambda w: w[1], reverse=True)[:topn]

## some example searches
print('Most similar French words for English words : computer, germany, matrix, physics, yeast \n',[mostSimilar(envec[e], frvec, 1) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']] )
print('Most similar Portugese words for English words : computer, germany, matrix, physics, yeast \n',[mostSimilar(envec[e], ptvec, 1) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']] )

Most similar French words for English words : computer, germany, matrix, physics, yeast 
 [[('informatique', 0.5023827767603765)], [('allemagne', 0.593718413875964)], [('matrice', 0.5088361302065517)], [('physique', 0.4555543434796394)], [('fermentation', 0.3504105196166514)]]
Most similar Portugese words for English words : computer, germany, matrix, physics, yeast 
 [[('computador', 0.4988965701007087)], [('alemanha', 0.6288992279001664)], [('matrix', 0.42047042528306583)], [('astrofísica', 0.5025296240252473)], [('fermentação', 0.4029791881209132)]]


Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [11]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2022-04-09 22:15:50--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab [following]
--2022-04-09 22:15:50--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.1’

links.tab.1         100%[===================>]   1.34M  --.-KB/s    in 0.1s    

2022-04-09 22:15:50 (10.5 MB/s) - ‘links.tab.1’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [12]:
links[200]

['abner', 'pt', 'abner']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [13]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
total = 0
acc = 0
base = 0

for row in links:
    if row[1] == 'fr':
        if row[0] in envec.keys():
            total += 1
            if row[0] == row[2]:
                base += 1
            similar = mostSimilar(envec[row[0]], frvec, 1)
            if similar[0][0] == row[2]:
                acc += 1

accuracy = acc/total
baselineAccuracy = base/total
print('English-French Wikipedia retrieval, Accuracy : {}, Baseline Accuracy : {}'.format(accuracy, baselineAccuracy))

English-French Wikipedia retrieval, Accuracy : 0.5359205593271862, Baseline Accuracy : 0.6742324450298915


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [14]:
## TODO: Compute English-Portuguese Wikipedia retrieval accuracy.
total = 0
acc = 0
base = 0

for row in links:
    if row[1] == 'pt':
        if row[0] in envec.keys():
            total += 1
            if row[0] == row[2]:
                base += 1
            similar = mostSimilar(envec[row[0]], ptvec, 1)
            if similar[0][0] == row[2]:
                acc += 1

accuracy = acc/total
baselineAccuracy = base/total
print('English-Portuguese Wikipedia retrieval, Accuracy : {}, Baseline Accuracy : {}'.format(accuracy, baselineAccuracy))

English-Portuguese Wikipedia retrieval, Accuracy : 0.49465079937492484, Baseline Accuracy : 0.5266257963697559


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [15]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

# Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors of each English term in French
total1 = 0
reciprocal1 = 0
recall1 = 0

total2 = 0
reciprocal2 = 0
recall2 = 0

for row in links:
    if row[1] == 'fr':
        if row[0] in envec.keys():
            total1 += 1
            similar = mostSimilar(envec[row[0]], frvec, 10)
            for i in range(len(similar)):
              if similar[i][0] == row[2]:
                recall1 += 1
                rank = i+1
                reciprocal1 += 1/rank
                break
    if row[1] == 'pt':
        if row[0] in envec.keys():
            total2 += 1
            similar = mostSimilar(envec[row[0]], ptvec, 10)
            for i in range(len(similar)):
              if similar[i][0] == row[2]:
                recall2 += 1
                rank = i+1
                reciprocal2 += 1/rank
                break

recall1_10 = recall1/total1
mrr1_10 = reciprocal1/total1
print('English-French Wikipedia retrieval, Recall@10 : {}, MRR@10 : {} '.format(recall1_10, mrr1_10))

recall2_10 = recall2/total2
mrr2_10 = reciprocal2/total2
print('English-Portuguese Wikipedia retrieval, Recall@10 : {}, MRR@10 : {} '.format(recall2_10, mrr2_10))

English-French Wikipedia retrieval, Recall@10 : 0.6130307021988043, MRR@10 : 0.5646970954423574 
English-Portuguese Wikipedia retrieval, Recall@10 : 0.5904555836037986, MRR@10 : 0.5301355951283833 


The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.